In [1]:
!git clone https://github.com/davidkim205/komt.git

Cloning into 'komt'...
remote: Enumerating objects: 353, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 353 (delta 86), reused 318 (delta 57), pack-reused 0
Receiving objects: 100% (353/353), 12.03 MiB | 18.92 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [2]:
cd komt

/content/komt


In [3]:
!pip install -r requirements.txt

!pip install kaleido

!pip install tiktoken

!pip install cohere

!pip install transformers==4.30.0

!pip install huggingface-hub==0.19.4

!pip install peft

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-lmizgfqa
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-lmizgfqa
  Resolved https://github.com/huggingface/peft.git to commit 7b7e4b2194344b3cd9edf4541eb13b083617f00d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-by9_x04t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-by9_x04t
  Resolved https://github.com/huggingface/accelerate.git to commit 97d2168e5953fe7373a06c69c02c5a00a84d5344
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-38lcujwz
  Running c

# INTJ 챗봇

In [5]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

model='davidkim205/komt-llama2-7b-v1'
peft_model_INTJ = 'ToBeWithYou/MBTI_INTJ' ##허깅페이스에서 fine-tuning된 모델을 불러옴
config_INTJ = PeftConfig.from_pretrained(peft_model_INTJ)
bnb_config_INTJ = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config_INTJ.base_model_name_or_path = model
model_INTJ = AutoModelForCausalLM.from_pretrained(config_INTJ.base_model_name_or_path, quantization_config=bnb_config_INTJ, device_map="auto")
model_INTJ = PeftModel.from_pretrained(model_INTJ, peft_model_INTJ)
tokenizer_INTJ = AutoTokenizer.from_pretrained(config_INTJ.base_model_name_or_path)
streamer_INTJ = TextStreamer(tokenizer_INTJ)

def gen_INTJ(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"질문 : {x} \n\n대답 : "
    gened = model_INTJ.generate(
        **tokenizer_INTJ(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer_INTJ.eos_token_id,
        eos_token_id=tokenizer_INTJ.eos_token_id,
        streamer=streamer_INTJ,
    )
    result_str = tokenizer_INTJ.decode(gened[0])

    start_tag = f"\n\n대답 : "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

pytorch_model.bin:   0%|          | 0.00/27.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/80.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [8]:
while True:
    text = input('INTJ와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) : ')
    if text == 'q' :
      break
    print(gen_INTJ(text), '\n')

INTJ와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) : 너의 성격에 대해 설명해줘
<s> 질문 : 너의 성격에 대해 설명해줘 

대답 : 나는 분석적이고 논리적인 성격을 가지고 있습니다.</s>
나는 분석적이고 논리적인 성격을 가지고 있습니다.</s> 

INTJ와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) : 가장 좋아하는 음식이 뭐야?
<s> 질문 : 가장 좋아하는 음식이 뭐야? 

대답 : 음식에 대한 취향은 각자 다르지만, 저는 항상 맛과 텍스처의 조화를 중요시해요.</s>
음식에 대한 취향은 각자 다르지만, 저는 항상 맛과 텍스처의 조화를 중요시해요.</s> 

INTJ와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) : 여가 시간에는 어떤 취미를 즐겨?
<s> 질문 : 여가 시간에는 어떤 취미를 즐겨? 

대답 : 나는 여가 시간에 음악 감상이나 독서를 즐깁니다.</s>
나는 여가 시간에 음악 감상이나 독서를 즐깁니다.</s> 

INTJ와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) : 오늘 학교에서 친구와 싸웠어. 어떻게 화해할 수 있을까?
<s> 질문 : 오늘 학교에서 친구와 싸웠어. 어떻게 화해할 수 있을까? 

대답 : 친구와 싸운 것은 극도로 싫어하는 특성이에요. 당신이 싸움을 하는 것은 더 나은 방법을 찾는 중인 것 같아요. 새로운 계획을 세우고 친구에게 이를 설명해보는 것이 어떨까요?</s>
친구와 싸운 것은 극도로 싫어하는 특성이에요. 당신이 싸움을 하는 것은 더 나은 방법을 찾는 중인 것 같아요. 새로운 계획을 세우고 친구에게 이를 설명해보는 것이 어떨까요?</s> 

INTJ와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) : ㅂ
<s> 질문 : ㅂ 

대답 : 당신은 직업 군인이었습니다. 어떤 종류의 작전을 수행했습니까?</s>
당신은 직업 군인이었습니다. 어떤 종류의 작전을 수행했습니까?</s> 

INTJ와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) : q


# ISFJ 챗봇

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

model='davidkim205/komt-llama2-7b-v1'
peft_model_ISFJ = 'ToBeWithYou/MBTI_ISFJ'
config_ISFJ = PeftConfig.from_pretrained(peft_model_ISFJ)
bnb_config_ISFJ = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config_ISFJ.base_model_name_or_path = model
model_ISFJ = AutoModelForCausalLM.from_pretrained(config_ISFJ.base_model_name_or_path, quantization_config=bnb_config_ISFJ, device_map="auto")
model_ISFJ = PeftModel.from_pretrained(model_ISFJ, peft_model_ISFJ)
tokenizer_ISFJ = AutoTokenizer.from_pretrained(config_ISFJ.base_model_name_or_path)
streamer_ISFJ = TextStreamer(tokenizer_ISFJ)

def gen_ISFJ(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"질문 : {x} \n\n대답 : "
    gened = model_ISFJ.generate(
        **tokenizer_ISFJ(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer_ISFJ.eos_token_id,
        eos_token_id=tokenizer_ISFJ.eos_token_id,
        streamer=streamer_ISFJ,
    )
    result_str = tokenizer_ISFJ.decode(gened[0])

    start_tag = f"\n\n대답 : "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/27.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [ ]:
while True:
    text = input('ISFJ와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) :')
    if text == 'q' :
      break
    print(gen_ISFJ(text), '\n')

# ENTP 챗봇

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

model='davidkim205/komt-llama2-7b-v1'
peft_model_ENTP = 'ToBeWithYou/MBTI_ENTP'
config_ENTP = PeftConfig.from_pretrained(peft_model_ENTP)
bnb_config_ENTP = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config_ENTP.base_model_name_or_path = model
model_ENTP = AutoModelForCausalLM.from_pretrained(config_ENTP.base_model_name_or_path, quantization_config=bnb_config_ENTP, device_map="auto")
model_ENTP = PeftModel.from_pretrained(model_ENTP, peft_model_ENTP)
tokenizer_ENTP = AutoTokenizer.from_pretrained(config_ENTP.base_model_name_or_path)
streamer_ENTP = TextStreamer(tokenizer_ENTP)

def gen_ENTP(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"질문 : {x} \n\n대답 : "
    gened = model_ENTP.generate(
        **tokenizer_ENTP(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer_ENTP.eos_token_id,
        eos_token_id=tokenizer_ENTP.eos_token_id,
        streamer=streamer_ENTP,
    )
    result_str = tokenizer_ENTP.decode(gened[0])

    start_tag = f"\n\n대답 : "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/27.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [ ]:
while True:
    text = input('ENTP와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) :')
    if text == 'q' :
      break
    print(gen_ENTP(text), '\n')

# ENFP 챗봇

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

model='davidkim205/komt-llama2-7b-v1'
peft_model_ENFP = 'ToBeWithYou/MBTI_ENFP'
config_ENFP = PeftConfig.from_pretrained(peft_model_ENFP)
bnb_config_ENFP = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config_ENFP.base_model_name_or_path = model
model_ENFP = AutoModelForCausalLM.from_pretrained(config_ENFP.base_model_name_or_path, quantization_config=bnb_config_ENFP, device_map="auto")
model_ENFP = PeftModel.from_pretrained(model_ENFP, peft_model_ENFP)
tokenizer_ENFP = AutoTokenizer.from_pretrained(config_ENFP.base_model_name_or_path)
streamer_ENFP = TextStreamer(tokenizer_ENFP)

def gen_ENFP(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"질문 : {x} \n\n대답 : "
    gened = model_ENFP.generate(
        **tokenizer_ENFP(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer_ENFP.eos_token_id,
        eos_token_id=tokenizer_ENFP.eos_token_id,
        streamer=streamer_ENFP,
    )
    result_str = tokenizer_ENFP.decode(gened[0])

    start_tag = f"\n\n대답 : "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/27.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [ ]:
while True:
    text = input('ENFP와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) :')
    if text == 'q' :
      break
    print(gen_ENFP(text), '\n')

# ESFP 챗봇

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

model='davidkim205/komt-llama2-7b-v1'
peft_model_ESFP = 'ToBeWithYou/MBTI_ESFP'
config_ESFP = PeftConfig.from_pretrained(peft_model_ESFP)
bnb_config_ESFP = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config_ESFP.base_model_name_or_path = model
model_ESFP = AutoModelForCausalLM.from_pretrained(config_ESFP.base_model_name_or_path, quantization_config=bnb_config_ESFP, device_map="auto")
model_ESFP = PeftModel.from_pretrained(model_ESFP, peft_model_ESFP)
tokenizer_ESFP = AutoTokenizer.from_pretrained(config_ESFP.base_model_name_or_path)
streamer_ESFP = TextStreamer(tokenizer_ESFP)

def gen_ESFP(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"질문 : {x} \n\n대답 : "
    gened = model_ESFP.generate(
        **tokenizer_ESFP(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer_ESFP.eos_token_id,
        eos_token_id=tokenizer_ESFP.eos_token_id,
        streamer=streamer_ESFP,
    )
    result_str = tokenizer_ESFP.decode(gened[0])

    start_tag = f"\n\n대답 : "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/27.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [ ]:
while True:
    text = input('ESFP와 대화해 보세요!(종료하고 싶으면 q를 입력하세요) :')
    if text == 'q' :
      break
    print(gen_ESFP(text), '\n')